# THE WHAT?

Where shall agile go?

In [1]:
file = '../../data/papersummaries_cleaned.csv'

In [2]:
import pandas as pd
import re

In [175]:
df = pd.read_csv(file)

NB: Because 'validate' is also used for the meta studies (validation of some procedure), we will need to remove 'bad tags' later

In [176]:
def helper(regex):
    res = [x for x in df.PaperSummary if re.search(regex, x)]
    return (len(res), res)

def give(loc):
    return df.PaperSummary[loc]

def match(loc):
    summary = give(loc)
    return (summary, assign_all_what(summary))

### The Rules

In [177]:
def assign_tag(level_1, level_2, level_3, summary):
    tags = []
    for tag in level_2:
        if any([re.search(keyword, summary) for keyword in level_3[tag]]):
            tags.append(level_1+tag)
    return tags

In [438]:
def assign_phase(summary):
    level_1 = ':phase'
    
    level_2 = [':elicitation', ':analysis', ':specification', ':evaluation', ':management']
    
    level_3 = {':elicitation':
                   ['[Ee]licit', '[Dd]iscover']
               , 
               ':analysis':
                   ['[Aa]naly[sz]ing\srequirements', 'requirements\sanalysis']
               , 
               ':specification':
                   ['[Rr]equirements\sspec', 'specify']
               ,
               ':evaluation':
                   ['[Vv]erif', '[Vv]alidat']
               ,
               ':management':
                   ['[Mm]aint[ae]', '\s[Aa]dapt', '\s[Rr]eus[ei]']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [419]:
def assign_quality(summary):
    level_1 = ':information:quality'
    
    level_2 = ['_compatibility', '_maintainability', '_performance', 
               '_portability', '_reliability', '_safety', '_security', '_usability', '_all']
    
    level_3 = {'_compatibility':   ['[Cc]ompatib'], 
               '_maintainability': [], 
               '_performance':     [],
               '_portability': ['[Pp]ortab'],
               '_reliability': ['reliab', 'dependab'],
               '_safety':      ['[Ss]afety\s'],
               '_security':    ['[Ss]ecur'],
               '_usability':   ['[Uu]sability'],
               '_all': ['[Qq]uality\s[Rr]eq', 'non-functional']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [397]:
def assign_information(summary):
    level_1 = ':information:quality'
    
    level_2 = ['_goals', '_systembehavior']
    
    level_3 = {'_goals': ['stakeholder\sgoals']
               , 
               '_systembehavior': ['behaviou?ral\sreq']
               }
    
    return assign_tag(level_1, level_2, level_3, summary) + assign_quality(summary)

In [386]:
def assign_documentation(summary):
    level_1 = ':documentation'
    
    level_2 = [':diagrams', ':businessmodels', ':goalmodels', 
               ':naturallanguage', ':prototypes', ':usecases', ':userstories']
    
    level_3 = {':diagrams':
                   ['[Dd]iagram']
               , 
               ':businessmodels':
                   []
               , 
               ':goalmodels':
                   ['goal\smodel']
               ,
               ':naturallanguage':
                   ['natural\slanguage']
               ,
               ':prototypes':
                   ['[Pp]rototyp']
               ,
               ':usecases':
                   ['use\scase']
               ,
               ':userstories':
                   ['user\sstories']
               ,
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [442]:
def assign_challenge_content(summary):
    level_1 = ':challenge:content'
    
    level_2 = ['_unambiguousness', '_consistency', 
               '_feasibility', '_traceability', 
               '_completeness', '_understandability'
              ]
    
    level_3 = {'_unambiguousness':
                   ['ambigu', 'clarif']
               ,
               '_consistency':
                   ['inconsisten']
               ,
               '_feasibility':
                   []
               ,
               '_traceability':
                   ['\strac']
               ,
               '_completeness':
                   ['completeness', 'incomplete', 'insufficient', 'missing\srequire']
               ,
               '_understandability':
                   ['understandability', 'comprehens(?:ion|ibility)']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [443]:
def assign_challenge_people(summary):
    level_1 = ':challenge:people'
    
    level_2 = ['_communication', '_collaboration', '_perception', '_belief', '_skills'
              ]
    
    level_3 = {'_communication':
                   ['[Cc]ommunica', 'misunderst']
               ,
               '_collaboration':
                   ['[Cc]ollabor']
               ,
               '_perception':
                   ['[Pp]ercepti']
               ,
               '_belief':
                   ['[Bb]elie[fv]']
               ,
               '_skills':
                   ['educat', '(?<!machine\s)learning', 'skills', 'teaching', 'creativ']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [389]:
def assign_challenge_process(summary):
    level_1 = ':challenge:process'
    
    level_2 = ['_automation', '_formalization', '_standardization'
               , '_prioritization', '_visualization', '_deciding'
              ]
    
    level_3 = {'_automation':
                   ['[Aa]utomat(?:ic|ion|ing|es|e)'],
               '_formalization':
                   ['[Ff]ormaliz'],
               '_prioritization':
                   ['[Pp]riorit'],
               '_standardization':
                   ['[Ss]tandardiz', '[Tt]emplate', '[Rr]eus[ea]', '[Bb]lueprint'],
               '_visualization':
                   ['[Vv]isual'],
               '_deciding':
                   ['[Dd]ecision[- ]making']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [429]:
def assign_challenge_context(summary):
    level_1 = ':challenge:context'
    
    level_2 = ['_uncertainty', '_regulation',
              ]
    
    level_3 = {'_uncertainty':
                   ['uncertain'],
               '_regulation':
                   ['regulat[io]', 'jurisdict']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [331]:
def assign_domain_class(summary):
    level_1 = ':domain:systemclass'
    
    level_2 = ['_embedded', '_safetycritical', '_adaptive', '_complex']
    
    level_3 = {'_embedded': ['[Ee]mbedded\s[Ss]ystem'],
               '_safetycritical': ['[Aa]utomotive', '[Ss]ubsea\s[Ee]quipment'],
               '_adaptive': ['[Aa]daptive\s[Ss]ystem'],
               '_complex': ['[Cc]omplex.*?[Ss]ystem']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

def assign_domain_sector(summary):
    level_1 = ':domain:sector'
    
    level_2 = ['_automotive', '_subsea', '_mobile', '_web']
    
    level_3 = {'_automotive': ['[Aa]utomotive'],
               '_subsea': ['[Ss]ubsea'],
               '_mobile': ['[Mm]obile'],
               '_web': ['\s[Ww]eb\s']
               }
    
    return assign_tag(level_1, level_2, level_3, summary)

In [391]:
def assign_region(summary):
    level_1 = ':region'
    
    level_2 = [':continent_europe', ':continent_asia', ':continent_namerica', 
               ':continent_samerica', ':continent_africa', ':continent_australia',
               ':country_finland', ':country_china'
              ]
    
    level_3 = {x:[] for x in level_2
               }
    
    level_3[':country_finland'] = ['Finland']
    level_3[':country_russia'] = ['Russia']
    
    return assign_tag(level_1, level_2, level_3, summary)

In [398]:
def assign_all_what(summary):
    tags = ['what'+ x for x in 
            (
                assign_phase(summary) # add further things here
                + assign_information(summary)
                + assign_documentation(summary)
                + assign_challenge_content(summary)
                + assign_challenge_people(summary)
                + assign_challenge_process(summary)
                + assign_challenge_context(summary)
                + assign_domain_class(summary)
                + assign_domain_sector(summary)
            )]
    return tags

In [399]:
match(157)

('A modelling language for modelling the behavioural requirements of a software product line in order to ease the task of adding new features to a set of existing requirements.',
 ['what:information:quality_systembehavior'])

In [437]:
helper('specify')

(11,
 ['A method for developing scenarios at multiple abstractions levels and check their consistency in order to specify complex software-intensive systems',
  'A method for specifying confidentiality requirements in service level agreements in order to provide proof of being in control of outsourced IT assets',
  'A method for using integer constraint programming to specify product line constraints in order to facilitate the configuration of a product',
  'A modeling language for specifying the system and finding criteria to compare candidate solutions at the early phases of requirements engineering',
  'A method for specifying and validating feature-oriented requirement models in order to avoid behavioral inconsistencies between features.',
  'A method for eliciting quality-impact relationships and using them to specify quality requirements in order to achieve good-enough quality.',
  'A modeling language for defining non-functional requirements (NFRs) based on the fulfillment of ot

In [417]:
#[assign_all_what(x) for x in df.PaperSummary]

In [444]:
df['what'] = [assign_all_what(x) for x in df.PaperSummary]

In [445]:
df.head()

,PaperID,PaperSummary,what
0,1,A method for automatically recovering software...,"[what:challenge:content_traceability, what:cha..."
1,2,A set of two techniques for improving the qual...,"[what:challenge:content_traceability, what:cha..."
2,3,A case study on evaluating a given technique f...,[what:challenge:process_prioritization]
3,4,An experience report on the development of a m...,"[what:phase:evaluation, what:challenge:process..."
4,5,A document-driven study on the relevancy of cl...,[what:phase:specification]


Summaries without tags?

In [447]:
len([(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) < 1])

115

In [446]:
[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) < 1]

[(20,
  'An interview-based study for explaining the causes of an unsatisfactory RE process in a company in order to identify improvement points',
  []),
 (22,
  'A method for systematically and repeatedly exchanging requirements between manufacturers and suppliers.',
  []),
 (23,
  'A method for formally representing requirements in order to facilitate their analysis and serve as basis for the testing process',
  []),
 (36,
  'A discussion on the need of future research on operationalizing requirements as runtime entities that can be reasoned over in order to support self-adaptation decisions',
  []),
 (37,
  'An experience report on the application of a requirements framework developed after a root cause analysis in order to enhance customer partnerships in an IT company',
  []),
 (44,
  'A case study on understanding the effects of system architectures on RE decisions, the characteristics of such decisions and their impact on development activities',
  []),
 (45,
  'An experience re

Summaries with multiple tags? (to check whether they're okay)

In [405]:
[(x[0], x[1][1], x[1][2]) for x in df.iterrows() if len(x[1][2]) > 1]

[(0,
  'A method for automatically recovering software traceability links between various software artifacts based on topic modelling (requirements, design, code, bug reports, test cases)',
  ['what:challenge:content_traceability',
   'what:challenge:process_automation']),
 (1,
  'A set of two techniques for improving the quality of traces generated between regulatory standards and product level requirements',
  ['what:challenge:content_traceability',
   'what:challenge:context_regulation']),
 (3,
  'An experience report on the development of a methodology and tools for the formalization and subsequent validation of specifications in a project for the public administration',
  ['what:phase:evaluation', 'what:challenge:process_formalization']),
 (8,
  'A tool for automating ambiguity detection and explaining the sources of the detected ambiguities in order to improve natural language specifications and educate software analysts',
  ['what:documentation:naturallanguage',
   'what:challen

In [275]:
df

,PaperID,PaperSummary,what
0,1,A method for automatically recovering software...,"[what:challenge:content_traceability, what:cha..."
1,2,A set of two techniques for improving the qual...,[what:challenge:content_traceability]
2,3,A case study on evaluating a given technique f...,[what:challenge:process_prioritization]
3,4,An experience report on the development of a m...,"[what:phase:evaluation, what:challenge:process..."
4,5,A document-driven study on the relevancy of cl...,[]
5,6,A case study on the effects of a highly ambigu...,[what:challenge:content_unambiguousness]
6,7,A data-driven study on the importance of quali...,[]
7,8,An interview-based study with practitioners fo...,[what:phase:evaluation]
8,9,A tool for automating ambiguity detection and ...,"[what:documentation:naturallanguage, what:chal..."
9,10,An experiment with practitioners for evaluatin...,[what:phase:management]


In [276]:
longform_tags = pd.DataFrame(columns=['PaperID', 'Tag'])
for row in df.iterrows():
    longform_tags = longform_tags.append(
        pd.DataFrame({'PaperID':list(len(row[1][2])*[row[1][0]]), 'Tag':row[1][2]}))
    #longform_tags = longform_tags.append(
        #pd.DataFrame({'PaperID':list(len(row[1][3])*[row[1][0]]), 'Tag':row[1][3]}))
longform_tags = longform_tags.reset_index().drop('index', axis=1)
longform_tags.head()

,PaperID,Tag
0,1,what:challenge:content_traceability
1,1,what:challenge:process_automation
2,2,what:challenge:content_traceability
3,3,what:challenge:process_prioritization
4,4,what:phase:evaluation


In [277]:
longform_tags.shape

(376, 2)

In [278]:
tags_four_levels = list(zip(*[x.split(':') for x in longform_tags.Tag]))
longform_tags['level_1'] = tags_four_levels[0]
longform_tags['level_2'] = tags_four_levels[1]
longform_tags['level_3'] = [x.split('_')[0] for x in tags_four_levels[2]]
longform_tags['level_4'] = [x.split('_')[-1] if (len(x.split('_')) > 1) 
                                                    else '' for x in tags_four_levels[2]]
longform_tags.head()

,PaperID,Tag,level_1,level_2,level_3,level_4
0,1,what:challenge:content_traceability,what,challenge,content,traceability
1,1,what:challenge:process_automation,what,challenge,process,automation
2,2,what:challenge:content_traceability,what,challenge,content,traceability
3,3,what:challenge:process_prioritization,what,challenge,process,prioritization
4,4,what:phase:evaluation,what,phase,evaluation,


NB: some papers appear multiple times in the counts (here: twice at max) since some papers receive multiple tags.

In [279]:
longform_tags.groupby(['level_1', 'level_2', 'level_3', 'level_4']).count()[['PaperID']]

PaperID
level_1 level_2       level_3         level_4                   
what    challenge     content         completeness             5
                                      consistency              8
                                      traceability            41
                                      unambiguousness         14
                                      understandability        4
                      context         uncertainty             11
                      people          belief                   1
                                      collaboration            9
                                      communication           12
                                      knowledge                2
                                      perception               3
                      process         automation              47
                                      formalization            1
                                      prioritization          15
                                      standardization          7
                                      visualization           12
        documentation diagrams                                 4
                      goalmodels                               6
                      naturallanguage                         29
                      prototypes                               1
                      usecases                                10
                      userstories                              4
        domain        sector          automotive               4
                                      mobile                   4
                                      subsea                   1
                                      web                      3
                      systemclass     adaptive                 6
                                      embedded                 2
                                      safetycritical           5
        information   quality         safety                   6
                                      security                19
                                      usability                2
        phase         analysis                                 6
                      elicitation                             35
                      evaluation                              23
                      management                              14

The End.